In [ ]:
from io import BytesIO

import pandas as pd
import numpy as np
import requests

from bokeh.plotting import figure, show, output_notebook
from bokeh.palettes import viridis
from bokeh.models import LabelSet, ColumnDataSource

output_notebook()

In [ ]:
def download_data(spreadsheet_url):
    """
    Download the spreadsheet with the swords data, and convert it to a dataframe.
    
    The spreadsheet should have the following columns:
        sword_name: the name of the measured sword for the current row
        cm: the position of the measurement, as cm from the pommel
        g: the weight of the sword at that position
        notes: is that point special?
    """
    
    response = requests.get(spreadsheet_url)
    assert response.status_code == 200, 'Unable to download swords data'

    data = pd.read_csv(BytesIO(response.content))

    return data

In [ ]:
swords = download_data('http://goo.gl/wbJNoa')

In [ ]:
def graph_swords(swords, only=None, show_labels=False):
    """Graph the swords measurements as lines."""
    f = figure(title='Effective mass', width=900, y_range=(0, 2000))
    
    if only:
        swords_to_graph = only
    else:
        swords_to_graph = swords.sword.unique()
        
    palette = viridis(len(swords_to_graph))
        
    for i, sword in enumerate(swords_to_graph):
        sword_data = swords[swords.sword == sword]
        f.line(sword_data.cm.values, 
               sword_data.g.values, 
               legend=sword, 
               line_color=palette[i],
               line_width=1)
        
        sword_poi = sword_data.dropna()
        f.circle(sword_poi.cm.values, 
                 sword_poi.g.values, 
                 legend=None,
                 color=palette[i],
                 size=7)
        
        if show_labels:
            source = ColumnDataSource(data=sword_poi)
            labels = LabelSet(source=source, 
                              x='cm', y='g', text='notes',
                              x_offset=5, y_offset=5)
            f.add_layout(labels)
        
    f.xaxis.axis_label = 'cm'
    f.yaxis.axis_label = 'g'
    
    show(f)    

In [ ]:
graph_swords(swords)

In [ ]:
graph_swords(swords, only=['katana_dan'], show_labels=True)